## Das Module multiprocessing

Erstellen Sie zwei parallelisierte Implementierung ihrer $\pi$-Bistimmung nach Monte Carlo, wovon eine auf *map* und eine auf *Queues* basiert. Vergleichen Sie die Rechenzeiten mit denen ihrer bisherigen Version. Wurden Ihre Erwartungen bezüglich der Beschleunigung erfüllt?

In [9]:
%load_ext cython

In [10]:
%%cython -a
from libc.stdlib cimport rand, srand, RAND_MAX
cimport cython

cpdef monte_carlo(quantity, seed):
    return cdef_monte_carlo(quantity, seed)

@cython.cdivision(True)
cdef int cdef_monte_carlo(int quantity, int seed):
    cdef int hits = 0
    cdef double width = 0.0
    cdef double height = 0.0
    cdef double temp = 0.0
    srand(seed)
    
    for _ in range(quantity):
        width = <double>rand()/RAND_MAX
        height = <double>rand()/RAND_MAX
        if width * width + height * height < 1:
            hits += 1

    return hits

In [11]:
#Monte Carlo Test
import random

random_number = int(random.random() * (2**31 -1))
trys = 25_000_000
print(monte_carlo(trys, random_number) / trys * 4)

3.14196912


In [12]:
%%timeit
monte_carlo(10_000, random_number)

257 µs ± 675 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
def worker_monte_carlo(q_in, q_out):
    while True:
        arguments = q_in.get()
        x, y = arguments
        result = monte_carlo(x,y)
        q_out.put(result)
        q_in.task_done()

In [ ]:
import multiprocessing
import math
import random
nrOfCores = multiprocessing.cpu_count()
print('nrOfCores:', nrOfCores)

l = []
for i in range (128):
    random_number = int(random.random() * (2**31 -1))
    l.append((2_500_000, random_number))

In [ ]:
print(l)

In [ ]:
in_queue = multiprocessing.JoinableQueue()
result_queue = multiprocessing.Queue()

processes = []
for i in range(nrOfCores):
    p = multiprocessing.Process(target = worker_monte_carlo, 
                                args = (in_queue, result_queue))
    processes.append(p)
    p.start()
    
for parameter_set in l:
    in_queue.put(parameter_set)
    
import time

in_queue.join()

result_list = []
while not result_queue.empty():
    result_list.append(result_queue.get())

print(result_list)

hits = 0
trys = 0

for e in l:
    x, y = e
    trys += x

for result_tuple in result_list:
    hits += result_tuple

    
print((hits)/trys * 4)
for p in processes:
    p.terminate()

## TimeIt

In [ ]:
%%timeit

import multiprocessing
in_queue = multiprocessing.JoinableQueue()
result_queue = multiprocessing.Queue()

processes = []
for i in range(nrOfCores):
    p = multiprocessing.Process(target = worker_monte_carlo, 
                                args = (in_queue, result_queue))
    processes.append(p)
    p.start()
    
for parameter_set in l:
    in_queue.put(parameter_set)
    
import time

in_queue.join()

result_list = []
while not result_queue.empty():
    result_list.append(result_queue.get())

print(result_list)

hits = 0
trys = 0

for e in l:
    x, y = e
    trys += x

for result in result_list:
    hits += result
    
print((hits)/trys * 4)
for p in processes:
    p.terminate()
    
#5.96 s ± 33.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#6.03 s ± 70.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#5.9 s ± 38.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

#without hypot
#1.94 s ± 5.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

## Map

In [ ]:
%%timeit
x = [e[0] for e in l]
y = [e[1] for e in l] 
result_list = map(monte_carlo, x, y)

hits = 0
trys = 0

for e in l:
    x, y = e
    trys += x

for result in result_list:
    hits += result
    
print(result_list)
print((hits)/trys * 4)

#23.9 s ± 205 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#23.5 s ± 86.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
%%timeit
process_pool = multiprocessing.Pool(processes = nrOfCores)
result_list = process_pool.starmap(monte_carlo, l)
process_pool.close()

hits = 0
trys = 0

for e in l:
    x, y = e
    trys += x

for result in result_list:
    hits += result
    
print(result_list)
print((hits)/trys * 4)

#5.88 s ± 130 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#5.81 s ± 49.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#5.81 s ± 24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#5.74 s ± 29.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

## Numba JIT Test

In [1]:
from numba import jit
import random

@jit
def monte_carlo(quantity, seed):
    random.seed(seed)
    hits = 0
    width = 0.0
    height = 0.0
    temp = 0.0
    
    for _ in range(quantity):
        width = random.random()
        height = random.random()
        if width * width + height * height < 1:
            hits += 1

    return hits

In [2]:
trys = 2_500_000
print(monte_carlo(trys, 5123432) / trys * 4)

3.1412768


In [7]:
%%timeit

import multiprocessing
import math
import random
nrOfCores = multiprocessing.cpu_count()
print('nrOfCores:', nrOfCores)

l = []
for i in range (128):
    random_number = int(random.random() * (2**31 -1))
    l.append((250_000_000, random_number))

process_pool = multiprocessing.Pool(processes = nrOfCores)
result_list = process_pool.starmap(monte_carlo, l)
process_pool.close()

hits = 0
trys = 0

for e in l:
    x, y = e
    trys += x

for result in result_list:
    hits += result
    
print(result_list)
print((hits)/trys * 4)

#5.88 s ± 130 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#5.81 s ± 49.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#5.81 s ± 24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#5.74 s ± 29.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

nrOfCores: 8
[196345499, 196339652, 196344956, 196335015, 196346429, 196353082, 196351251, 196347791, 196364703, 196351077, 196353628, 196347633, 196350367, 196358393, 196352855, 196349409, 196351266, 196346620, 196357560, 196344137, 196338554, 196340417, 196342069, 196344475, 196362417, 196344526, 196350385, 196347944, 196349925, 196356147, 196347608, 196333338, 196351814, 196344042, 196362845, 196342476, 196350220, 196343537, 196341833, 196348873, 196351359, 196352627, 196345454, 196346963, 196357493, 196344192, 196348951, 196353336, 196343447, 196347808, 196350162, 196353670, 196335656, 196346075, 196337579, 196347719, 196352420, 196355300, 196350314, 196352157, 196352413, 196356912, 196338267, 196344918, 196350553, 196357291, 196353474, 196362184, 196357212, 196360966, 196346295, 196347564, 196353309, 196345595, 196345040, 196345374, 196351612, 196356332, 196349876, 196336176, 196351205, 196343270, 196346979, 196360479, 196350482, 196347233, 196346524, 196354644, 196347853, 1963524

[196348667, 196350633, 196354556, 196348856, 196359148, 196359313, 196351246, 196344756, 196349648, 196355733, 196363028, 196362062, 196336211, 196367071, 196354633, 196361193, 196352543, 196341340, 196345309, 196369035, 196341573, 196356847, 196345400, 196350254, 196353449, 196355082, 196359705, 196355794, 196351927, 196343132, 196341943, 196349728, 196338211, 196356269, 196343915, 196333726, 196347771, 196364234, 196355033, 196358155, 196343567, 196345025, 196351676, 196348342, 196342720, 196353652, 196343232, 196350338, 196345107, 196347921, 196343294, 196349973, 196359920, 196355877, 196344789, 196341406, 196343172, 196351095, 196346769, 196362593, 196347361, 196354316, 196358409, 196349164, 196347416, 196358246, 196355236, 196356659, 196355394, 196349849, 196350184, 196359713, 196353776, 196346356, 196352606, 196343524, 196344311, 196347423, 196350313, 196350067, 196350464, 196365340, 196346504, 196349299, 196356551, 196359564, 196354044, 196353848, 196342390, 196340240, 196342263